# Access a MISP instance using pymisp

Before running this, you'll need:

* Pip install pymisp
* Access token for a MISP instance

More help at
* https://pypi.org/project/pymisp/
* Pymisp notebooks: https://github.com/MISP/PyMISP/tree/master/docs/tutorial 

In [1]:
from pymisp import ExpandedPyMISP


# Get MISP creds
cscsecret = open('/Users/sara/cscmisp.txt', 'r')
(cscurl,csckey) = [x.strip() for x in cscsecret.readline().split(',')]
cscsecret.close()
misp = ExpandedPyMISP(cscurl, csckey, True)

# Uncomment if you want to see what misp can do
# help(misp)

# Read in an incident and its details

What we'd like to do is 1) see all the MISP events that we have for our group
* Get a list of events for our group - or all connected groups
* Filter that list by topics of interest (e.g. COVID-19 tags)
* See the event ids that correspond to the incident names we use in our other systems

We'd like to read data on a specific event, so we can use it in analysis
* Get data for a named incident

We'd also like to know about the data that's connected to a specific event, so we can analyse that too
* Get objects connected to a named incident

In [8]:
# List the Covid19 results we have in the MISP
results = misp.search("events", tags="COVID-19")
eventids = {result['Event']['info']:result['Event']['id'] for result in results}
eventids

{'US Piracy': '31', 'Covid5G': '8090'}

In [38]:
# We have 2 ways to view data for an event - we can uncomment the next line, and 
# look at it directly
# results[1]['Event']

# Or we can get the event by id / name; directly with "misp.get_event(8090)" or with this
# Have separated out the keys - these are basically flat except for 
# * Org/Orgc (details of organisation)
# * Galaxy
# * Object
# * Tag
eventname = 'Covid5G'
if eventname in eventids:
    eventdetails = misp.get_event(eventids[eventname]) 
    print('Keys are: {}\n'.format(eventdetails['Event'].keys()))
    print('Tags are: {}\n'.format([x['name'] for x in eventdetails['Event']['Tag']]))
    print('TTPs are: {}\n'.format([[x['value'] for x in y['GalaxyCluster']] for y in eventdetails['Event']['Galaxy']]))
    print('Objects are: {}\n'.format([(x['name'], x['description']) for x in eventdetails['Event']['Object']]
))
    print('Details are: {}'.format(eventdetails))

Keys are: dict_keys(['id', 'orgc_id', 'org_id', 'date', 'threat_level_id', 'info', 'published', 'uuid', 'attribute_count', 'analysis', 'timestamp', 'distribution', 'proposal_email_lock', 'locked', 'publish_timestamp', 'sharing_group_id', 'disable_correlation', 'extends_uuid', 'event_creator_email', 'Org', 'Orgc', 'Attribute', 'ShadowAttribute', 'RelatedEvent', 'Galaxy', 'Object', 'Tag'])

Tags are: ['COVID-19', 'misp-galaxy:amitt-misinformation-pattern="Create hashtag"', 'misp-galaxy:amitt-misinformation-pattern="Cultivate useful idiots"', 'misp-galaxy:amitt-misinformation-pattern="Conspiracy narratives"', 'misp-galaxy:amitt-misinformation-pattern="Twitter"', 'misp-galaxy:amitt-misinformation-pattern="Use hashtag"']

TTPs are: [['Create hashtag', 'Cultivate useful idiots', 'Conspiracy narratives', 'Twitter', 'Use hashtag']]

Objects are: [('microblog', 'Microblog post like a Twitter tweet or a post on a Facebook wall.')]

Details are: {'Event': {'id': '8090', 'orgc_id': '1', 'org_id': 

# Create an incident in MISP

What we'd like to do is 1) add a new incident to MISP
* set distribution to "connected communities"
* set threat level to undefined
* set event info to the name of the incident
* set tags to e.g. "Covid-19"
* (maybe) add the misinformation pattern - although that's probably better done by hand
* get the event id for this incident

What we'd like to do is also 2) add mass incident data to this event from a spreadsheet
* Add an object to the event for every microblog (tweet or facebook post)
* Add an object for every facebook group
* Add an object for every URL
* Fill out as many details as we can for each type of object

What we'd also like to do is 3) add incident data to this event from slack, via bot
* as above, but using bot commands

In [ ]:
from datetime import date
from pymisp import MISPEvent

#fill out these things
incidentstart = '2020-04-14' #date.today() 
incidentname = 'operationgridlock'
tags = ['tlp:white', 'COVID-19', 'lockdown']

# Create event object
event = MISPEvent()

event.info = incidentname
event.distribution = 2
event.threat_level_id = 4
event.analysis = 0 #initial analysis
event.set_date(incidentstart)
for tag in tags:
    event.add_tag(tag)

newevent = misp.add_event(event, pythonify=True)

print(event.to_json())

# Add an object to a MISP incident

In [51]:
from pymisp import MISPObject
from misp_objects import MicroblogObject

# Create object
object_properties = {'url': 'https://twitter.com/josh_emerson/status/1250539109246001152?s=19'}
misp_object = MicroblogObject(parameters=object_properties)

# Add object to incident
incidentid = str(8090)
incidentevent = misp.get_event(incidentid, pythonify=True)
incidentevent.add_object(misp_object)
response = incidentevent.Object.append(misp_object)
response = misp.update_event(incidentid, incidentevent)